<a href="https://colab.research.google.com/github/Hyperdraw/ai-workshop-project/blob/dev/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Workshop Project

In [1]:
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import LSTM

Sequential model

In [2]:
def cur_model(num_Conv2D,num_filters,num_kernel,num_lstm_units,num_frames):
  model = Sequential()
  for i in range(num_Conv2D):
    model.add(Conv2D(num_filters, num_kernel, activation ='relu'))
  model.add(Reshape((num_frames, -1)))
  model.add(LSTM(num_lstm_units))

In [3]:
def reinforcement(env, model, n_games=1, epochs=1, epsilon_decrease=0.1, backsight=8):
  epsilon = 1
  state_history = []
  value_history = []
  obs = env.observation_space.sample()

  for game_number in range(n_games):
    states = []
    actions = []
    values = []
    memory = [obs] * backsight
    frames = 0

    while True:
      if random.rand() > epsilon:
        action = np.argmax(model.predict(np.array([memory]))[0])
      else:
        action = env.action_space.sample()

      actions.append(action)
      states.append(memory)
      values.append([0] * env.action_space.n)
      obs, reward, done = env.step(action)
      memory.pop(0)
      memory.append(obs)

      for i, scores in enumerate(values):
        scores[actions[i]] += reward
      
      frames += 1
      
      if done:
        print('Game', game_number, 'lasted', frames, 'frames')
        break
    
    state_history.extend(states)
    value_history.extend(values)
    model.fit(np.array(state_history), np.array(value_history))
    epsilon = max(0, epsilon - epsilon_decrease)